In [ ]:
import os
import sys

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

matplotlib.style.use("Solarize_Light2")

%matplotlib inline

In [ ]:
def check_null(df):
    """
    Returns percentage of rows containing missing data
    """
    return df.isna().sum() * 100/len(df)


def get_missing_dates(series, start_date, end_date, freq="D"):
    """
    Returns the dates which are missing in the series
    date_sr between the start_date and end_date
    
    series: Series consisting of date
    start_date: Start date in String format
    end_date: End date in String format
    """
    return pd.date_range(
        start=start_date, end=end_date, freq=freq).difference(series)


def check_duplicate(df, subset):
    """
    Returns if there are any duplicate rows in the DataFrame.
    
    df: DataFrame under consideration
    subset: Optional List of feature names based on which 
            duplicate rows are being identified. 
    """
    if subset is not None: 
        return df.duplicated(subset=subset, keep=False).sum()
    else:
        return df.duplicated(keep=False).sum()


def create_date_features(source_df, target_df, feature_name):
    '''
    Create new features related to dates
    
    source_df : DataFrame consisting of the timestamp related feature
    target_df : DataFrame where new features will be added
    feature_name : Name of the feature of date type which needs to be decomposed.
    '''
    target_df.loc[:, 'year'] = source_df.loc[:, feature_name].dt.year.astype('uint16')
    target_df.loc[:, 'month'] = source_df.loc[:, feature_name].dt.month.astype('uint8')
    target_df.loc[:, 'quarter'] = source_df.loc[:, feature_name].dt.quarter.astype('uint8')
    target_df.loc[:, 'weekofyear'] = source_df.loc[:, feature_name].dt.isocalendar().week.astype('uint8')
    
    target_df.loc[:, 'hour'] = source_df.loc[:, feature_name].dt.hour.astype('uint8')
    
    target_df.loc[:, 'day'] = source_df.loc[:, feature_name].dt.day.astype('uint8')
    target_df.loc[:, 'dayofweek'] = source_df.loc[:, feature_name].dt.dayofweek.astype('uint8')
    target_df.loc[:, 'dayofyear'] = source_df.loc[:, feature_name].dt.dayofyear.astype('uint8')
    target_df.loc[:, 'is_month_start'] = source_df.loc[:, feature_name].dt.is_month_start
    target_df.loc[:, 'is_month_end'] = source_df.loc[:, feature_name].dt.is_month_end
    target_df.loc[:, 'is_quarter_start']= source_df.loc[:, feature_name].dt.is_quarter_start
    target_df.loc[:, 'is_quarter_end'] = source_df.loc[:, feature_name].dt.is_quarter_end
    target_df.loc[:, 'is_year_start'] = source_df.loc[:, feature_name].dt.is_year_start
    target_df.loc[:, 'is_year_end'] = source_df.loc[:, feature_name].dt.is_year_end
    
    # This is of type object
    target_df.loc[:, 'month_year'] = source_df.loc[:, feature_name].dt.to_period('M')
    
    return target_df


def plot_boxh_groupby(df, feature_name, by):
    """
    Box plot with groupby
    
    df: DataFrame
    feature_name: Name of the feature to be plotted
    by: Name of the feature based on which groups are created
    """
    df.boxplot(column=feature_name, by=by, vert=False, 
                              figsize=(10, 6))
    plt.title(f'Distribution of {feature_name} by {by}')
    plt.show()
    

def plot_hist(df, feature_name, kind='hist', bins=100, log=True):
    """
    Plot histogram.
    
    df: DataFrame
    feature_name: Name of the feature to be plotted.
    """
    if log:
        df[feature_name].apply(np.log1p).plot(kind='hist', 
                                              bins=bins, 
                                              figsize=(15, 5), 
                                              title=f'Distribution of log1p[{feature_name}]')
    else:
        df[feature_name].plot(kind='hist', 
                              bins=bins, 
                              figsize=(15, 5), 
                              title=f'Distribution of {feature_name}')
    plt.show()


def plot_ts(series, figsize=(20, 6), title=None, xlabel="", ylabel=""):
    """
    Plot Time Series data. The series object should have date or time as index.
    
    series: Series object to be plotted.
    """
    series.plot(figsize=figsize, title=title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()


def plot_barh(df, feature_name, normalize=True, 
              kind='barh', figsize=(15,5), sort_index=False, title=None):
    """
    Plot barh for a particular feature
    
    kind : Type of the plot
    
    """
    if sort_index==True:
        df[feature_name].value_counts(
                normalize=normalize, dropna=False).sort_index().plot(
                kind=kind, figsize=figsize, grid=True,
                title=title)
    else:   
        df[feature_name].value_counts(
                normalize=normalize, dropna=False).sort_values().plot(
                kind=kind, figsize=figsize, grid=True,
                title=title)
    
    plt.legend()
    plt.show()


def plot_boxh(df, feature_name, kind='box', log=True):
    """
    Box plot
    """
    if log:
        df[feature_name].apply(np.log1p).plot(kind='box', vert=False, 
                                                  figsize=(10, 6), 
                                                  title=f'Distribution of log1p[{feature_name}]')
    else:
        df[feature_name].plot(kind='box', vert=False, 
                              figsize=(10, 6), 
                              title=f'Distribution of {feature_name}')
    plt.show()
    

def plot_scatter(df, feature_x, feature_y, figsize=(10,10), 
                 title=None, xlabel=None, ylabel=None):
    """
    Plot satter     
    """
    df.plot.scatter(feature_x, feature_y, 
                    figsize=(8, 6), title=title, 
                    legend=None)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()

# Exploratory Data Analysis for Predictive Maintenance (PdM)


### This notebook explores the data related to Predictive Maintenance ([link](https://www.kaggle.com/arnabbiswas1/microsoft-azure-predictive-maintenance)) provided by Microsoft Azure.


# What is Predictive Maintenance?

Our world is full of equipment. For example:
* Aircraft consists of different equipments
* HVAC (Heating, Ventilation, Air Conditioning) equipment consists of various parts

All these equipments or their parts meet failures and hence need maintenance. Thus maintenance is a big industry by itself. Maintenance is done either by replacing parts at regular intervals even when those are working (**Preventive Maintenance**) or by replacing the parts only when there is failure (**Reactive Maintenance**). **Predictive Maintenance** avoids the drawbacks of Preventive Maintenance (under utilization of a part's life) and Reactive Maintenance (unscheduled downtime). Based on the health of an equipment in the past, future point of failure can be predicted in Predictive Maintenance. Thus, replacement of parts can be scheduled just before the actual failure. 

Traditionally, predictive maintenance is being done using rule based techniques. With the advent of connected sensors (IoT), data from equipment is continuously collected and fed to Machine Learning based systems to predict its future health.

# Data Description

There are 5 CSV files consisting of:

- **Telemetry Time Series Data** (PdM_telemetry.csv): It consists of hourly average of voltage, rotation, pressure, vibration collected from 100 machines for the year 2015.

- **Error** (PdM_errors.csv): These are errors encountered by the machines while in operating condition. Since, these errors don't shut down the machines, these are not considered as failures. The error date and times are rounded to the closest hour since the telemetry data is collected at an hourly rate.

- **Maintenance** (PdM_maint.csv): If a component of a machine is replaced, that is captured as a record in this table. Components are replaced under two situations:
    - During the regular scheduled visit, the technician replaced it (Proactive Maintenance)
    - A component breaks down and then the technician does an unscheduled maintenance to replace the component (Reactive  Maintenance). This is considered as a failure and corresponding data is captured under Failures.
    Maintenance data has both 2014 and 2015 records. This data is rounded to the closest hour since the telemetry data is collected at an hourly rate.
    
    

- **Failures** (PdM_failures.csv): Each record represents replacement of a component due to failure. This data is a subset of Maintenance data. This data is rounded to the closest hour since the telemetry data is collected at an hourly rate.

- **Metadata of Machines** (PdM_Machines.csv): Model type & age of the Machines.

In [ ]:
# Read the data
DATA_DIR = "/kaggle/input/microsoft-azure-predictive-maintenance/"

telemetry_df = pd.read_csv(f"{DATA_DIR}/PdM_telemetry.csv")
errors_df = pd.read_csv(f"{DATA_DIR}/PdM_errors.csv")
maint_df = pd.read_csv(f"{DATA_DIR}/PdM_maint.csv")
failures_df = pd.read_csv(f"{DATA_DIR}/PdM_failures.csv")
machines_df = pd.read_csv(f"{DATA_DIR}/PdM_machines.csv")

# Format date & time. Sort based on date for better readability
tables = [telemetry_df, maint_df, failures_df, errors_df]
for df in tables:
    df["datetime"] = pd.to_datetime(df["datetime"], format="%Y-%m-%d %H:%M:%S")
    df.sort_values(["datetime", "machineID"], inplace=True, ignore_index=True)


# Telemetry Data

### This data consists of hourly average of voltage, rotation, pressure, vibration collected from 100 machines for the year 2015.

In [ ]:
print(f"Shape of the Telemetry Records: {telemetry_df.shape}")
print("\n")
telemetry_df.head()

### Display the first few rows of the Telemetry data for Machine 1

In [ ]:
telemetry_df[telemetry_df.machineID == 1].head()

### How many Machines are there?

In [ ]:
telemetry_df.machineID.nunique()

### What is the duration of the data?

In [ ]:
telemetry_df.datetime.describe(datetime_is_numeric=True)

Telemetry data is distributed between 1st Jan 2015 to 1st Jan 2016

It seems that the data is having hourly frequency.

Let's check if there are missing rows.

### Are there any missinge days in the data?

In [ ]:
get_missing_dates(telemetry_df.datetime, 
                  start_date="2015-01-01 06:00:00", 
                  end_date="2016-01-01 06:00:00", 
                  freq='H')

There is no missing data.

### Are there any duplicates?

One Machine should not have multiple rows with the same time stamp.

In [ ]:
check_duplicate(telemetry_df, ['datetime', 'machineID'])

There are no duplicates in the telemetry data.

### Are there any Null values in the data?

In [ ]:
check_null(telemetry_df)

There are no missing values in the data

### Let's plot Vibrarion of Machine 1 for 2015

In [ ]:
# Let's plot Vibrarion of Machine 1 for 2015
df_vib_machine_1 = telemetry_df[
    telemetry_df.machineID == 1][["datetime", "vibration"]].set_index("datetime")
plot_ts(df_vib_machine_1, title="Vibration of Machine 1", xlabel="Time", ylabel="Vibration")

### Let's plot voltage of Machine 2 for 1st two weeks of 2015

In [ ]:
# Let's plot voltage of Machine 2 for 1st two weeks of 2015
df_vib_machine_1 = telemetry_df[
    (telemetry_df.machineID == 2) & (
        telemetry_df.datetime.dt.isocalendar().week.isin(
            [1, 2, 3]))][["datetime", "volt"]].set_index("datetime")
plot_ts(df_vib_machine_1, title="Volatage of Machine 2", xlabel="Time", ylabel="Voltage")

### Let's add date related features to the telemetry data

In [ ]:
telemetry_df = create_date_features(telemetry_df, telemetry_df, "datetime")
telemetry_df.head()

### Plot the distribution of voltage across various months. Ideally there should be some amount seasonality in the data

In [ ]:
plot_boxh_groupby(telemetry_df, feature_name="volt", by="month_year")

It shows the voltage across Machines are not varying over month. 

We can ignore the entry for 2016 since we only have data for one day in 2016.

#### Let's plot it just for Machine 80.

In [ ]:
plot_boxh_groupby(telemetry_df[telemetry_df.machineID == 80], feature_name="volt", by="month_year")

This kind of shows the same thing. There is not much variation of voltage across months.

### Let's plot the distribution of Voltage across Machines.

In [ ]:
plot_hist(telemetry_df, feature_name="volt", log=False, bins=1000)

Wow! The distribution is a perfect normal curve. **This indicates the possibility of synethetically generated data.**

Let's verify it by plotting histogram of other parameters.

In [ ]:
for name in ['rotate', 'pressure', 'vibration']:
    plot_hist(telemetry_df, feature_name=name, log=False, bins=1000)

Vibration, rotation and pressure are also normally distributed. This verifies what we suspected.

#### This data may be normally distributed. :-)

## Observations about Telemetry Data
- This may be synthetically generated data distributed between 1st Jan 2015 to 1st Jan 2016.
- Each row represents the state of a machine on a particular hour. Voltage, vibration, pressure & rotation of a machine have been averaged hourly.
- There are 100 unique Machines.
- There are no duplicates or missing values in the dataset.
- The four parameters voltage, vibration, pressure & rotation are normally distributed.

# Error Data

This data includes the errors encountered by the machines while in operating condition. Since, these errors don't shut down the machines, these are not considered as failures. The error date and times are rounded to the closest hour since the telemetry data is collected at an hourly rate.

In [ ]:
print(f"Shape of the Error Records: {errors_df.shape}")
print("\n")
errors_df.head()

In [ ]:
# Sort the Error based "datetime", "machineID", "errorID" for better readability
errors_df = errors_df.sort_values(["datetime", "machineID", "errorID"]).reset_index(drop=True)

There are 3919 errors in total.

In [ ]:
errors_df.machineID = errors_df.machineID.astype('category')
errors_df.errorID = errors_df.errorID.astype('category')

### What is the duration of the data?

In [ ]:
errors_df.datetime.describe(datetime_is_numeric=True)

Error data is distributed between 1st Jan 2015 to 1st Jan 2016.

Also data is captured in hourly fashion.

### Are there any duplicates?

One Machine should not have multiple errors with the same time stamp.

In [ ]:
check_duplicate(errors_df, ['datetime', 'machineID', 'errorID'])

There are no duplicates in the error data.

### Are there any Null values in the data?

In [ ]:
check_null(errors_df)

### Plot different types of errors

In [ ]:
plot_barh(errors_df, 
          feature_name="errorID", 
          figsize=(10, 6), 
          normalize=False,
          title="Different Types of Errors")

Type 1 & 2 errors are most frequent

### Let's check if the failures are uniformly occuring across machines.

In [ ]:
plot_barh(errors_df, "machineID", figsize=(6, 20), normalize=False, title="Number of errors across MachineID")

As expected, different machines has encountered, different number of errors.

### How does the Machine to type of error distribution looks like?

In [ ]:
df_errors = errors_df.groupby(["machineID", "errorID"]).size().reset_index()
df_errors.columns = ["machineID", "errorID", "errorValues"]
#df_errors_pivot = pd.pivot(df_errors, index="machineID", columns="errorID", values="errorValues").reset_index().rename_axis(None, axis=1)
df_errors_pivot = pd.pivot(df_errors, index="machineID", columns="errorID", values="errorValues").rename_axis(None, axis=1)

df_errors_pivot.plot.bar(stacked=True, figsize=(20, 6), title="Count of Errors for different Machines")
plt.xlabel("Machine ID")
plt.ylabel("Number of Errors");

### Plot number of errors across Machines over days

In [ ]:
plot_ts(
    errors_df.datetime.dt.date.value_counts().sort_index(), 
    figsize=(20, 6), 
    title="Number of Errors Across Days", 
    xlabel="Time",
    ylabel="Number of Errors")

### How does the error distribution looks for a particluar machine?

In [ ]:
df_temp = errors_df[errors_df.machineID.isin([1, 2])].datetime.dt.date.value_counts().sort_index()
df_temp.plot(style="k.", figsize=(8, 4), title="Number of Errors Across Days for Machine 1 & 2")
plt.ylabel("Count of Errors")
plt.xlabel("Time")
plt.show()

For Machine 1 & 2, for most of the days, number of error is 1. But there are few days when number of errors are more than 1.

### Let's plot the distribution of the number of errors per day across Machine.

In [ ]:
errors_df['date'] = errors_df.datetime.dt.date

errors_df.groupby('date').size().hist(bins=20, figsize=(10, 6))
plt.title("Distribution of Number of Errors Per Day")
plt.xlabel("Number of Errors on a Particular Day")
plt.ylabel("Frequency")
plt.show()

# Maintenance Data

If a component of a machine is replaced, that is captured as a record in this table. Components are replaced under two situations:

- During the regular scheduled visit, the technician replaced it (Proactive Maintenance)
- A component breaks down and then the technician does an unscheduled maintenance to replace the component (Reactive Maintenance). This is considered as a failure and corresponding data is captured under Failures. Maintenance data has both 2014 and 2015 records. This data is rounded to the closest hour since the telemetry data is collected at an hourly rate.


In [ ]:
print(f"Shape of the Maintenance Records: {maint_df.shape}")
print("\n")
maint_df.head()

In [ ]:
for name in ["machineID", "comp"]:
    maint_df[name] = maint_df[name].astype("category")
    
maint_df.sort_values(["datetime", "machineID", "comp"], inplace=True)

# Add date related features.
maint_df = create_date_features(maint_df, maint_df, "datetime")

### What is the duration of the data?

In [ ]:
maint_df.datetime.describe(datetime_is_numeric=True)

Maintenance data is present June 2014 onwards. This is different from other data which are present between 2014 and 2015.

In [ ]:
get_missing_dates(maint_df.datetime, 
                  start_date="2014-06-01 06:00:00", 
                  end_date="2016-01-01 06:00:00", 
                  freq='H')

### Let's plot number of maintenance records across months

In [ ]:
plot_barh(maint_df, "month_year", normalize=False, sort_index=True)

Number of components replaced in the year 2015 are considerably higher compared to the 2014. 

Again, we can ignore the data for 2016 (since we have only one day's data)

### Let's check the number of components replaced

In [ ]:
plot_barh(maint_df, 
          feature_name="comp", 
          figsize=(10, 6), 
          normalize=False,
          title="Components Replaced")

Four types components are replaced almost in the same numbers.

### Let's plot the number of Maintenance Records Across Machines

In [ ]:
plot_barh(maint_df, "machineID", 
          figsize=(6, 20), 
          normalize=False, 
          title="Number of Maintenance Records across MachineID", 
          sort_index=True)

### How does the Machine to different component replaced looks like?

In [ ]:
df_maint = maint_df.groupby(["machineID", "comp"]).size().reset_index()
df_maint.columns = ["machineID", "comp", "num_comp"]
df_maint_pivot = pd.pivot(df_maint, index="machineID", columns="comp", values="num_comp").rename_axis(None, axis=1)

df_maint_pivot.plot.bar(stacked=True, figsize=(20, 6), title="Count of Components Replaced for different Machines")
plt.xlabel("Machine ID")
plt.ylabel("Number of Components Replaced");

### Plot number of Maintenance Issues reaised per day

In [ ]:
maint_df.datetime.dt.date.value_counts().plot(
    style="k.", 
    figsize=(20, 4), 
    title="Number of Maintenance Records Across Time")
plt.ylabel("Number of Maintenance Records")
plt.xlabel("Time")
plt.show()

This indicates that there is a drastic difference between the number of maintenance records in 2014 vs 2015.

# Failure Data

# Static Data of Machines

This data set includes some information about the machines: model type and age (years in service).

In [ ]:
print(f"Shape of the Machines Data: {machines_df.shape}")
print("\n")
machines_df.head()

### Plot the distribution of age of the Machines

In [ ]:
plot_boxh(machines_df, feature_name="age", log=False)

The age of the Machines is distributed between 0 to 20. The median age is to ~12.5. There are no outliers. Another indication that this is a synthetic data.

### Plot Age vs Number of Failures

In [ ]:
# Create a DF with number of errors, maintenance records and failure records across machines

# Create a DF consisting of number of erros across Machines
erros_across_machine = errors_df.groupby("machineID").size()
erros_across_machine = pd.DataFrame(erros_across_machine, columns=["num_errors"]).reset_index()

machines_errors_df = pd.merge(machines_df, erros_across_machine, how='left', on="machineID")

# Create a DF consisting of number of maintenance records across Machines
maint_across_machine = maint_df.groupby("machineID").size()
maint_across_machine = pd.DataFrame(maint_across_machine, columns=["num_maint"]).reset_index()

machines_errors_df = pd.merge(machines_errors_df, maint_across_machine, how='left', on="machineID")

# Create a DF consisting of number of failure records across Machines
failure_across_machine = failures_df.groupby("machineID").size()
failure_across_machine = pd.DataFrame(failure_across_machine, columns=["num_failure"]).reset_index()

machines_errors_df = pd.merge(machines_errors_df, failure_across_machine, how='left', on="machineID")

machines_errors_df.head()

### Plot Number of Errors across Machine Age

In [ ]:
plot_scatter(machines_errors_df, "age", "num_errors", 
             title="Age vs Number of Errors", 
             xlabel="Age", ylabel="Number of Errors")

### Plot Age vs Number of Maintenance Records

In [ ]:
plot_scatter(machines_errors_df, "age", "num_maint", 
             title="Age vs Number of Maintenance Records", 
             xlabel="Age", ylabel="Number of Maintenance Records")

### Plot Age vs Number of Failure Records

In [ ]:
plot_scatter(machines_errors_df, "age", "num_failure", 
             title="Age vs Number of Failure Records", 
             xlabel="Age", ylabel="Number of Failure Records")

#### From the above three plots, it appears only Number of Failures is slightly correlated with Age.

Let's verify it with a correlation values.

In [ ]:
machines_errors_df.corr()